In [ ]:
import torch
from transformers import ViTModel, ViTFeatureExtractor

device = 'cuda' if torch.cuda.is_available() else 'cpu'

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)
model.eval()
